In [1]:
import warnings
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

from itertools import islice, takewhile, chain
from functools import reduce
from typing import Optional
import datetime as dt
from dataclasses import asdict, fields
from importlib import reload

from geopy.distance import distance
from shapely.geometry import Point, LineString
import shapely.geometry as sg
import geopandas as gpd

from typing import List
import ipyleaflet as lf

pd.set_option('display.max_rows', 1000000000)

In [2]:
import busboy.model as m
import busboy.geo as geo
import busboy.database as db
import busboy.prediction as prediction
import busboy.map.map as bmap
import busboy.apis as api
import busboy.util as util
import busboy.util.notebooks as notebook

/Users/Noel/.local/share/virtualenvs/Busboy-8t8akAoa/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [41]:
reload(util)
reload(geo)
reload(m)
reload(db)
reload(prediction)
reload(bmap)
reload(api)
reload(notebook)

<module 'busboy.util.notebooks' from '/Users/Noel/Developer/Projects/Busboy/busboy/util/notebooks.py'>

In [44]:
rbn = db.routes_by_name()
route = rbn["201"].id
entries = db.snapshots(r=route, d=dt.date(2019, 2, 18))
stops_by_name = db.stops_by_name()
timetables = list(api.timetables("201", stops_by_name))
timetable_variants = {t for timetable in timetables for t in timetable.variants}

In [45]:
entries_by_vehicle = util.dict_collect_list(entries, lambda e: e.vehicle)
vehicles_by_entry_count = [v for (v, es) in sorted(entries_by_vehicle.items(), key = lambda t: len(t[1]), reverse=True)]

In [47]:
pvars = sorted(list(
    prediction.possible_variants(
        prediction.drop_duplicate_positions(entries_by_vehicle[vehicles_by_entry_count[0]]), 
        timetable_variants
    )),
    key = lambda t: t[0].poll_time)

In [48]:
order_pvars = list(prediction.check_variant_order(pvars))

In [49]:
route_sections = {v: list(prediction.route_sections(v.stops)) for v in timetable_variants}
stop_shaped_entries = [(e, {v: {t[1] for t in ts} for v, ts in util.dict_collect_set(vs, lambda tpl: tpl[0]).items()}) for (e, vs) in order_pvars]
times = prediction.stop_times(stop_shaped_entries, route_sections)
section_times = prediction.section_times(stop_shaped_entries, route_sections)
journeys = prediction.journeys(section_times)

In [53]:
to_time = lambda d: d.time().isoformat()
for variant, these_stop_times in times.items():
    print(f"Variant: {variant}")
    print(f"{len(these_stop_times)} journeys")
    for trip_number, trip_times in enumerate(these_stop_times):
        for stop_number, stop_time in enumerate(trip_times):
            to_time = lambda d: d.time().isoformat()
            try:
                stop_name = variant.stops[stop_number].name
            except IndexError:
                stop_name = "(IndexError)"
            print(f"- {trip_number}, {stop_number:2}, {stop_name:40}"
                  f" {stop_time.last_before.map(to_time).or_else(''):15}, {stop_time.first_after.map(to_time).or_else(''):15}")

Variant: (route: 201, start: Boherboy Rd (Opp Scoil Mhuire Banrion), end: CUH (Bishopstown Rd))
21 journeys
- 0,  0, Boherboy Rd (Opp Scoil Mhuire Banrion)                  , 07:19:32.536890
- 0,  1, Boherboy Road (Lotabeg Estate)           07:18:52.525808, 07:20:32.577308
- 0,  2, North Ring Road (Opp Mayfield Supermarke 07:20:12.559419, 07:21:32.628519
- 0,  3, North Ring Road (Lagan Grove)            07:21:32.628519, 07:22:52.652098
- 0,  4, North Ring Rd (Glencree Crescent)        07:22:52.652098, 07:23:32.658942
- 0,  5, North Ring Rd (Corrib Lawn)              07:23:32.658942, 07:24:32.690832
- 0,  6, North Ring Rd (Boyne Crescent)           07:24:32.690832, 07:25:12.698864
- 0,  7, North Ring Rd (Opp Riverview Estate)     07:25:52.734574, 07:27:32.789463
- 0,  8, Old Commons Rd (Opp Topaz Service St)    07:29:32.858933, 07:30:52.904094
- 0,  9, Farranferris Ave (Pophams Rd Junction)   07:30:52.904094, 07:32:32.993772
- 0, 10, Pophams Rd (Opposite Community Centre)   07:31:52.958

In [50]:
themap = bmap.Map()
themap.map

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [51]:
notebook.show_timetables(themap, timetables)

In [56]:
themap.map.add_control(lf.LayersControl())

In [52]:
[(v, len(entries_by_vehicle[v])) for v in vehicles_by_entry_count]

[(VehicleId(raw='7338674957838189376'), 7885), (VehicleId(raw=None), 658)]